# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans, MeanShift, AgglomerativeClustering, Birch, OPTICS, DBSCAN
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
from hdbscan import HDBSCAN
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster import cluster_visualizer
from pyclustering.utils import timedcall
from time import time
import memory_profiler

- This script performs a comparative analysis of various clustering algorithms on the California Housing dataset, utilizing PCA for dimension reduction to 2, 3, and 4 components.
- It includes:
  1. Two sets of clustering algorithms: traditional ones like K-Means, Mini Batch K-Means, and K-Medoids, and advanced ones like Mean Shift, Gaussian Mixture, and other clustering methods (Agglomerative Clustering, BIRCH, OPTICS, HDBSCAN, DBSCAN).
  2. Execution of each clustering algorithm on PCA-reduced data, measuring performance metrics such as Silhouette Score, Davies-Bouldin Index, and Calinski-Harabasz Index to evaluate cluster quality.
  3. Calculation of resource consumption for each algorithm, including time and memory usage, to assess efficiency.
- The results are collected in two separate DataFrames for detailed analysis and comparison of the effectiveness and efficiency of each algorithm across different PCA settings.

# Data Loading & Preprocessing

In [2]:
path = 'C:\\Users\\zcindemir\\Desktop\\Data Mining Project\\housing.csv'
housing = pd.read_csv(path)

housing['total_bedrooms'].fillna(housing['total_bedrooms'].median(), inplace=True)

encoder = OneHotEncoder()
ocean_proximity_encoded = encoder.fit_transform(housing[['ocean_proximity']]).toarray()
feature_names = encoder.get_feature_names_out(['ocean_proximity'])
ocean_proximity_df = pd.DataFrame(ocean_proximity_encoded, columns=feature_names)
housing_encoded = pd.concat([housing.drop('ocean_proximity', axis=1), ocean_proximity_df], axis=1)

scaler = StandardScaler()
housing_scaled = scaler.fit_transform(housing_encoded)

## K-means, K-Medoids, K-Medians and Mini Batch K-Means

In [3]:
pca_results = {}
for n_components in [2, 3, 4]:
    pca = PCA(n_components=n_components)
    pca_results[n_components] = pca.fit_transform(housing_scaled)

cluster_algorithms = {
    'K-Means': KMeans(n_clusters=6, n_init='auto'),
    'Mini Batch K-Means': MiniBatchKMeans(n_clusters=6, n_init='auto', batch_size=3072),
    'K-Medoids': KMedoids(n_clusters=6),
    'K-Medians': None
}

results = []
for n_components, data in pca_results.items():
    for algo_name, algo in cluster_algorithms.items():
        start_time = time()
        mem_usage_start = memory_profiler.memory_usage()

        if algo_name == 'K-Medians':
            initial_medians = [data[np.random.randint(0, len(data))].tolist() for _ in range(6)]
            kmedians_instance = kmedians(data, initial_medians)
            (ticks, _) = timedcall(kmedians_instance.process)
            clusters = kmedians_instance.get_clusters()
            labels = np.zeros(len(data))
            for cluster_id, cluster in enumerate(clusters):
                for index in cluster:
                    labels[index] = cluster_id
        else:
            labels = algo.fit_predict(data)

        mem_usage_end = memory_profiler.memory_usage()
        end_time = time()

        if np.unique(labels).size > 1:
            silhouette = metrics.silhouette_score(data, labels)
            db_index = metrics.davies_bouldin_score(data, labels)
            ch_index = metrics.calinski_harabasz_score(data, labels)
        else:
            silhouette, db_index, ch_index = np.nan, np.nan, np.nan
        
        time_consumed = end_time - start_time
        memory_consumed = mem_usage_end[0] - mem_usage_start[0]
        
        results.append({
            'PCA Components': n_components,
            'Algorithm': algo_name,
            'Silhouette Score': silhouette,
            'Davies-Bouldin Index': db_index,
            'Calinski-Harabasz Index': ch_index,
            'Time Consumed (s)': time_consumed,
            'Memory Consumed (MB)': memory_consumed
        })


results_df_1 = pd.DataFrame(results)

In [4]:
indexed_df_1 = results_df_1.set_index(['PCA Components', 'Algorithm']).sort_index()
indexed_df_1

Silhouette Score  Davies-Bouldin Index  \
PCA Components Algorithm                                                    
2              K-Means                     0.394311              0.842350   
               K-Medians                   0.383132              0.801364   
               K-Medoids                   0.384561              0.802897   
               Mini Batch K-Means          0.325900              0.979884   
3              K-Means                     0.389376              1.026925   
               K-Medians                   0.307257              1.221944   
               K-Medoids                   0.277851              1.129313   
               Mini Batch K-Means          0.341428              1.021910   
4              K-Means                     0.401001              0.887667   
               K-Medians                   0.335333              1.046658   
               K-Medoids                   0.343329              1.013551   
               Mini Batch K-Means          0.380396              0.976570   

                                   Calinski-Harabasz Index  Time Consumed (s)  \
PCA Components Algorithm                                                        
2              K-Means                        15899.367999           0.348015   
               K-Medians                      13780.585734           0.867870   
               K-Medoids                      13500.902674          33.880282   
               Mini Batch K-Means             15309.544182           0.344059   
3              K-Means                         9428.270264           0.263344   
               K-Medians                       8056.220836           0.925411   
               K-Medoids                       7270.842536          21.600055   
               Mini Batch K-Means              9541.037522           0.343117   
4              K-Means                         8630.736117           0.278317   
               K-Medians                       7017.596200           1.148998   
               K-Medoids                       7475.164667          17.114232   
               Mini Batch K-Means              8381.740696           0.363039   

                                   Memory Consumed (MB)  
PCA Components Algorithm                                 
2              K-Means                         3.550781  
               K-Medians                      23.269531  
               K-Medoids                       1.082031  
               Mini Batch K-Means              0.511719  
3              K-Means                         0.929688  
               K-Medians                       2.789062  
               K-Medoids                      -0.542969  
               Mini Batch K-Means              0.007812  
4              K-Means                         1.171875  
               K-Medians                       0.031250  
               K-Medoids                      -3.925781  
               Mini Batch K-Means              0.218750

## Mean Shift, Gaussian Mixture Model, Agglomerative Clustering, BIRCH, OPTICS, HDBSCAN, and DBSCAN

In [5]:
pca_results = {}
for n_components in [2, 3, 4]:
    pca = PCA(n_components=n_components)
    pca_results[n_components] = pca.fit_transform(housing_scaled)

cluster_algorithms = {
    'Mean Shift': MeanShift(bandwidth=4.026624),
    'Gaussian Mixture': GaussianMixture(n_components=2, covariance_type='spherical', random_state=42),
    'Agglomerative Clustering': AgglomerativeClustering(n_clusters=6, linkage='single', metric='euclidean'),
    'BIRCH': Birch(threshold=0.5, branching_factor=100, n_clusters=5),
    'OPTICS': OPTICS(min_samples=15, xi=0.05, min_cluster_size=55),
    'HDBSCAN': HDBSCAN(min_cluster_size=5, min_samples=10),
    'DBSCAN': DBSCAN(eps=0.5, min_samples=10)
}

results = []
for n_components, data in pca_results.items():
    for algo_name, algo in cluster_algorithms.items():
        start_time = time()
        mem_usage_start = memory_profiler.memory_usage()

        labels = algo.fit_predict(data)

        mem_usage_end = memory_profiler.memory_usage()
        end_time = time()

        if np.unique(labels).size > 1:
            silhouette = metrics.silhouette_score(data, labels)
            db_index = metrics.davies_bouldin_score(data, labels)
            ch_index = metrics.calinski_harabasz_score(data, labels)
        else:
            silhouette, db_index, ch_index = np.nan, np.nan, np.nan
        
        time_consumed = end_time - start_time
        memory_consumed = mem_usage_end[0] - mem_usage_start[0]
        
        results.append({
            'PCA Components': n_components,
            'Algorithm': algo_name,
            'Silhouette Score': silhouette,
            'Davies-Bouldin Index': db_index,
            'Calinski-Harabasz Index': ch_index,
            'Time Consumed (s)': time_consumed,
            'Memory Consumed (MB)': memory_consumed
        })


results_df_2 = pd.DataFrame(results)

In [6]:
indexed_df_2 = results_df_2.set_index(['PCA Components', 'Algorithm']).sort_index()
indexed_df_2

Silhouette Score  \
PCA Components Algorithm                                    
2              Agglomerative Clustering          0.856173   
               BIRCH                             0.553121   
               DBSCAN                            0.652755   
               Gaussian Mixture                  0.306959   
               HDBSCAN                           0.655634   
               Mean Shift                        0.840483   
               OPTICS                            0.843555   
3              Agglomerative Clustering          0.826745   
               BIRCH                             0.194931   
               DBSCAN                            0.568741   
               Gaussian Mixture                  0.569450   
               HDBSCAN                           0.612225   
               Mean Shift                        0.792123   
               OPTICS                            0.728976   
4              Agglomerative Clustering          0.810531   
               BIRCH                             0.219274   
               DBSCAN                            0.261405   
               Gaussian Mixture                  0.129408   
               HDBSCAN                           0.276326   
               Mean Shift                        0.773110   
               OPTICS                            0.071802   

                                         Davies-Bouldin Index  \
PCA Components Algorithm                                        
2              Agglomerative Clustering              0.184165   
               BIRCH                                 0.620969   
               DBSCAN                                1.110853   
               Gaussian Mixture                      0.928817   
               HDBSCAN                               1.821081   
               Mean Shift                            0.297960   
               OPTICS                                0.307048   
3              Agglomerative Clustering              0.198335   
               BIRCH                                 1.244309   
               DBSCAN                                1.002220   
               Gaussian Mixture                      0.797889   
               HDBSCAN                               1.970577   
               Mean Shift                            0.346679   
               OPTICS                                0.543309   
4              Agglomerative Clustering              0.198637   
               BIRCH                                 1.153139   
               DBSCAN                                1.467259   
               Gaussian Mixture                      1.294529   
               HDBSCAN                               1.195330   
               Mean Shift                            0.366508   
               OPTICS                                1.054939   

                                         Calinski-Harabasz Index  \
PCA Components Algorithm                                           
2              Agglomerative Clustering               140.744925   
               BIRCH                                 2786.190475   
               DBSCAN                                1215.766418   
               Gaussian Mixture                      6612.322513   
               HDBSCAN                                967.965316   
               Mean Shift                            1989.632787   
               OPTICS                                1805.292548   
3              Agglomerative Clustering               106.149913   
               BIRCH                                 3419.909668   
               DBSCAN                                3582.298645   
               Gaussian Mixture                      5644.241739   
               HDBSCAN                               1043.581384   
               Mean Shift                             834.486080   
               OPTICS                                2413.763371   
4              Agglomerative Clustering              